### import and implement model

In [1]:
# packages
import torch
import torch.nn as nn
from torch.nn import Conv2d, LeakyReLU, MaxPool2d, Linear# import them seperetly because I think its more readable
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split
import pandas as pd
import os 
# set wd
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/PyTorch/Fracture')

import data and load onto tensors

In [2]:
# image dataset class
class ImageDataset():
    def __init__(self, class_dir, img_dir): # load labels and the img 
        self.img_labels = pd.read_csv(class_dir)
        self.img_dir = img_dir

    def __len__ (self): # len of labels for image
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])  
        image = read_image(img_path)  # read image
        label = self.img_labels.iloc[idx, 1] # read label
        return image, label


In [3]:
# directories for classes and images
class_dir = './images/class_ids.csv' 
image_dir = './images/resize_data' 

# load dataset using made class function
data_set = ImageDataset(class_dir, image_dir) # create dataset

# set train and test set
train_size = int(.9 * len(data_set)) # using 90% for training, can introduce more testing samples if testing isnt robust enough
test_size = len(data_set) - train_size

# random split 
training, testing = random_split(data_set, [train_size, test_size])

# load the split data on the tensors
train_loader = DataLoader(training, batch_size=32, shuffle=True)
test_loader = DataLoader(testing, batch_size=32, shuffle=True)

In [4]:
print(torch.cuda.is_available()) # check if gpu is working correctly

True


model without transfer learning (will add just wanted to build one from stratch)

In [ ]:
# I chose to use a CNN for the image classifcation. 
# CNNs preform much better then feed forward networks for image classification tasks and are still easy to implement

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 1 input layer, to 32 filters, stride of one pixel, 3x3 kernal, padding = (kernal - 1)/2

        # 3 layers like this
        self.conv1 = Conv2d(in_channels=1, out_channels=32, stride=1, kernel_size=3, padding=1) 
        self.Lrelu1 = LeakyReLU() # better preformance on average compared to regular ReLu
        self.conv2 = Conv2d(in_channels=32, out_channels=32, stride=1, kernel_size=3, padding=1)
        self.Lrelu2 = LeakyReLU()
        self.maxpool1 = MaxPool2d(kernel_size = 2, stride = 2)

        # 3 layers like this
        self.conv3 = Conv2d(in_channels=32, out_channels=64, stride=1, kernel_size=3, padding=1) 
        self.Lrelu3 = LeakyReLU() # better preformance on average compared to regular ReLu
        self.conv4 = Conv2d(in_channels=64, out_channels=64, stride=1, kernel_size=3, padding=1)
        self.Lrelu4 = LeakyReLU()
        self.maxpool2 = MaxPool2d(kernel_size = 2, stride = 2)

        # 3 layers like this
        self.conv5 = Conv2d(in_channels=64, out_channels=128, stride=1, kernel_size=3, padding=1) 
        self.Lrelu5 = LeakyReLU() # better preformance on average compared to regular ReLu
        self.conv6 = Conv2d(in_channels=128, out_channels=128, stride=1, kernel_size=3, padding=1)
        self.Lrelu6 = LeakyReLU()
        self.maxpool3 = MaxPool2d(kernel_size = 2, stride = 2)

        # fully connected layers
        self.fc1 = Linear(100352, 1024) # 100352 = 128 * 28 * 28 | 28 = W/8
        self.relu1 = LeakyReLU()
        self.fc2 = Linear(1024, 512)
        self.relu2 = LeakyReLU()
        self.fc3 = Linear(512, 256)
        self.relu3 = LeakyReLU()
        self.fc4 = nn.Linear(256, 1) # one output

        self.dropout = nn.Dropout(p=0.2), # 20% chance a nueron would be dropped, this reduces overfitting making one neuron not resposnible for everything, also improves regualrization
        self.activation = nn.Sigmoid()

    def foward_pass(self, x):
        # Pass through Convolutional Block 1
        x = self.conv1(x)
        x = self.Lrelu1(x)
        x = self.conv2(x)
        x = self.Lrelu2(x)
        x = self.maxpool1(x)

        # Pass through Convolutional Block 2
        x = self.conv3(x)
        x = self.Lrelu3(x)
        x = self.conv4(x)
        x = self.Lrelu4(x)
        x = self.maxpool2(x)

        # Pass through Convolutional Block 3
        x = self.conv5(x)
        x = self.Lrelu5(x)
        x = self.conv6(x)
        x = self.Lrelu6(x)
        x = self.maxpool3(x)

        x = x.view(x.size(0), -1)

        # Pass through Fully Connected Layers
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)

        # Pass through the final Linear layer
        x = self.fc4(x)

        # Apply Dropout and Sigmoid activation
        x = self.dropout(x) # Apply dropout
        x = self.sigmoid(x) # Apply final sigmoid activation

        return x

        
